# 🤖 Proyecto Nandito — Integración de CrewAI con NANDA y AWS Bedrock

**Nandito** es un proyecto experimental de interoperabilidad entre agentes de inteligencia artificial.  
El objetivo principal es demostrar cómo un agente desarrollado con **CrewAI** puede conectarse a la red descentralizada **NANDA**, utilizando **AWS Bedrock** como backend de modelos fundacionales (Claude 3.5 / Haiku).  

Este agente actúa como una interfaz inteligente capaz de:
- Comprender y mejorar mensajes recibidos a través del protocolo NANDA.
- Usar **CrewAI** para interpretar peticiones complejas y generar respuestas breves y útiles.
- Operar bajo un entorno de prueba (sandbox) con integración directa a **AWS Bedrock**.

El proyecto forma parte de la iniciativa de exploración del *Internet de Agentes (Agentic Web)*.


In [26]:
import sys
import os, pathlib
import boto3
import subprocess, sys, time

## ⚙️ Configuración del entorno

Para ejecutar este proyecto es necesario instalar las dependencias principales:
- **CrewAI**: Framework de coordinación de agentes.
- **LiteLLM y boto3**: Acceso a modelos de AWS Bedrock.
- **NANDA Adapter**: Librería experimental de interoperabilidad.
- **mcp-use / mcp-utils**: Bridge necesario para comunicación MCP (Multi-Channel Protocol).

Además, se aplica un pequeño parche de salida estándar (`stdout.reconfigure`) para evitar errores en entornos Jupyter.


In [2]:
%pip install -q crewai litellm boto3 nanda-adapter mcp-use requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.6/472.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.7/338.7 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 8.0 MB/s 

In [1]:
if not hasattr(sys.stdout, "reconfigure"):
    setattr(sys.stdout, "reconfigure", lambda *a, **k: None)
print("stdout patch OK")


stdout patch OK


## 🧩 Creación del bridge MCP

El adaptador NANDA requiere una dependencia llamada `mcp_utils`, la cual actualmente no está publicada de forma oficial.  
Por ello, se crea un *shim* local que redirige la importación hacia el paquete `mcp_use`, que contiene la clase `MCPClient`.

Este paso garantiza la compatibilidad entre NANDA Adapter y CrewAI.


In [3]:

shim_dir = pathlib.Path('/mnt/data/mcp_utils')
shim_dir.mkdir(parents=True, exist_ok=True)
(shim_dir / '__init__.py').write_text('from mcp_use import MCPClient\n')
print('Shim mcp_utils listo en', shim_dir)

Shim mcp_utils listo en /mnt/data/mcp_utils


## 🔑 Carga de credenciales AWS

Para conectarse a AWS Bedrock, el proyecto utiliza credenciales almacenadas en el archivo `bedrock_keys.txt`.  
Este archivo contiene las siguientes variables:


In [5]:
with open("bedrock_keys.txt") as f:
    for line in f:
        key, value = line.strip().split("=", 1)
        os.environ[key] = value

print("Credenciales cargadas")

Credenciales cargadas


In [12]:
with open("bedrock_keys.txt") as f:
    for line in f:
        if "=" in line:
            key, value = line.strip().split("=", 1)
            os.environ[key] = value

if not os.getenv("AWS_REGION") and os.getenv("AWS_DEFAULT_REGION"):
    os.environ["AWS_REGION"] = os.getenv("AWS_DEFAULT_REGION")

print("Credenciales cargadas:")
print("AWS_REGION:", os.getenv("AWS_REGION"))
print("AWS_ACCESS_KEY_ID:", os.getenv("AWS_ACCESS_KEY_ID")[:6] + "..." if os.getenv("AWS_ACCESS_KEY_ID") else "falta")
print("AWS_SECRET_ACCESS_KEY:", "configurada" if os.getenv("AWS_SECRET_ACCESS_KEY") else "falta")


Credenciales cargadas:
AWS_REGION: us-east-1
AWS_ACCESS_KEY_ID: AKIAWT...
AWS_SECRET_ACCESS_KEY: configurada


Estas credenciales se cargan dinámicamente en las variables de entorno de Python (`os.environ`) para permitir que `boto3` autentique las llamadas a AWS Bedrock.


In [14]:
if "/mnt/data" not in os.sys.path:
    os.sys.path.insert(0, "/mnt/data")
os.environ["PYTHONPATH"] = "/mnt/data:" + os.environ.get("PYTHONPATH", "")
print("Entorno listo")

Entorno listo


## 🤝 Integración de CrewAI y NANDA

En esta sección se crea el archivo `server_nanda.py`, el cual define el agente y su comportamiento dentro del ecosistema NANDA.

El flujo principal incluye:
- **Definición del modelo LLM** con `CrewAI` y `AWS Bedrock`.
- **Creación de un agente (`helper`)** con rol y objetivo definidos.
- **Implementación de la función `crewai_improvement()`**, que ejecuta tareas de mejora de mensajes.
- **Inicialización de NANDA Adapter**, que habilita el puente de comunicación entre agentes.

El servidor se ejecutará en el puerto 6000 y permitirá recibir mensajes a través del endpoint `/api/send`.


In [15]:
from pathlib import Path
code = r'''
import os
from crewai import Agent, Task, Crew, LLM
from nanda_adapter import NANDA

os.environ.setdefault("AWS_REGION", "us-east-1")

llm = LLM(
    model="bedrock/anthropic.claude-3-haiku-20240307-v1:0",
    temperature=0.1,
    max_tokens=800
)

helper = Agent(
    role="Research Helper",
    goal="Responder de forma breve y clara.",
    backstory="Asistente que usa CrewAI y Bedrock para contestar preguntas.",
    llm=llm,
    verbose=False
)

def crewai_improvement(message_text: str) -> str:
    task = Task(
        description=f"Responde claramente a: {message_text}",
        expected_output="Respuesta breve y útil.",
        agent=helper
    )
    result = Crew(agents=[helper], tasks=[task]).kickoff()
    return str(result)

def main():
    os.environ.setdefault("UI_CLIENT_URL", "")
    os.environ.setdefault("HOST", "0.0.0.0")
    os.environ.setdefault("PORT", "6000")
    os.environ.setdefault("A2A_PORT", os.environ.get("PORT", "6000"))

    nanda = NANDA(crewai_improvement)
    nanda.start_server_api("", "localhost")

if __name__ == "__main__":
    main()
'''
Path("/mnt/data/server_nanda.py").write_text(code)
print("Escrito /mnt/data/server_nanda.py")


Escrito /mnt/data/server_nanda.py


In [18]:
!crewai create crew AI_Agent_Claude

Creating folder ai_agent_claude...
Cache expired or not found. Fetching provider data from the web...
Select a provider to set up:
1. openai
2. anthropic
3. gemini
4. nvidia_nim
5. groq
6. huggingface
7. ollama
8. watson
9. bedrock
10. azure
11. cerebras
12. sambanova
13. other
q. Quit
Enter the number of your choice or 'q' to quit: 9
Select a model to use for Bedrock:
1. bedrock/us.amazon.nova-pro-v1:0
2. bedrock/us.amazon.nova-micro-v1:0
3. bedrock/us.amazon.nova-lite-v1:0
4. bedrock/us.anthropic.claude-3-5-sonnet-20240620-v1:0
5. bedrock/us.anthropic.claude-3-5-haiku-20241022-v1:0
6. bedrock/us.anthropic.claude-3-5-sonnet-20241022-v2:0
7. bedrock/us.anthropic.claude-3-7-sonnet-20250219-v1:0
8. bedrock/us.anthropic.claude-3-sonnet-20240229-v1:0
9. bedrock/us.anthropic.claude-3-opus-20240229-v1:0
10. bedrock/us.anthropic.claude-3-haiku-20240307-v1:0
11. bedrock/us.meta.llama3-2-11b-instruct-v1:0
12. bedrock/us.meta.llama3-2-3b-instruct-v1:0
13. bedrock/us.meta.llama3-2-90b-instruct-v1

In [20]:
from crewai import Agent, Task, Crew, LLM

llm = LLM(
    model="bedrock/anthropic.claude-3-haiku-20240307-v1:0",
    temperature=0.1,
    max_tokens=800
)

helper = Agent(
    role="Research Helper",
    goal="Responder consultas breves con claridad.",
    backstory="Asistente pragmático que luego podrá colaborar con otros agentes.",
    llm=llm,
    verbose=True
)

task = Task(
    description="Explica en 2 oraciones qué es CrewAI.",
    expected_output="Explicación breve en español.",
    agent=helper,
)
result = Crew(agents=[helper], tasks=[task]).kickoff()
print("Prueba CrewAI OK:\n", result)

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Research Helper                                                                                         │
│                                                                                                                 │
│  Task: Explica en 2 oraciones qué es CrewAI.                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Research Helper                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  CrewAI es una inteligencia artificial (IA) diseñada para colaborar con equipos humanos en proyectos y tareas.  │
│  Utiliza tecnologías de procesamiento de lenguaje natural y aprendizaje automático para entender las            │
│  necesidades del equipo, asignar tareas, coordinar esfuerzos y ofrecer asistencia en tiempo real.               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): Y


╭─────────────────────────────────────────── Trace Batch Finalization ────────────────────────────────────────────╮
│ ✅ Trace batch finalized with session ID: 75b59939-e32b-4be4-ac53-f013781dc072                                  │
│                                                                                                                 │
│ 🔗 View here:                                                                                                   │
│ https://app.crewai.com/crewai_plus/ephemeral_trace_batches/75b59939-e32b-4be4-ac53-f013781dc072?access_code=TRA │
│ CE-7e69d3f94e                                                                                                   │
│ 🔑 Access Code: TRACE-7e69d3f94e                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────── 🔍 Execution Trace Generated ──────────────────────────────────────────╮
│                                                                                                                 │
│  🎉 Your First CrewAI Execution Trace is Ready!                                                                 │
│                                                                                                                 │
│  View your execution details here:                                                                              │
│  https://app.crewai.com/crewai_plus/ephemeral_trace_batches/75b59939-e32b-4be4-ac53-f013781dc072?access_code=T  │
│  RACE-7e69d3f94e                                                                                                │
│                                                                                                                 │
│  This trace shows:                                                                                              │
│  • Agent decisions and interactions                                                                             │
│  • Task execution timeline                                                                                      │
│  • Tool usage and results                                                                                       │
│  • LLM calls and responses                                                                                      │
│                                                                                                                 │
│  ✅ Tracing has been enabled for future runs! (CREWAI_TRACING_ENABLED=true added to .env)                       │
│  You can also add tracing=True to your Crew(tracing=True) / Flow(tracing=True) for more control.                │
│                                                                                                                 │
│  📝 Note: This link will expire in 24 hours.                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Prueba CrewAI OK:
 CrewAI es una inteligencia artificial (IA) diseñada para colaborar con equipos humanos en proyectos y tareas. Utiliza tecnologías de procesamiento de lenguaje natural y aprendizaje automático para entender las necesidades del equipo, asignar tareas, coordinar esfuerzos y ofrecer asistencia en tiempo real.


In [21]:
from nanda_adapter import NANDA

def crewai_improvement(message_text: str) -> str:
    t = Task(
        description=f"Responde de forma clara a: {message_text}",
        expected_output="Una respuesta breve y útil.",
        agent=helper,
    )
    out = Crew(agents=[helper], tasks=[t]).kickoff()
    return str(out)

nanda = NANDA(crewai_improvement)

🤖 NANDA initialized with custom improvement logic: crewai_improvement
🔧 Custom improvement logic 'crewai_improvement' registered
Message improver set to: nanda_custom
✅ AgentBridge created with custom improve_message_direct: crewai_improvement


## 🚀 Ejecución del servidor NANDA

Una vez configurado el servidor, se lanza como un proceso independiente utilizando `subprocess.Popen`.  
Esto evita los errores de `signal` que se producen si se ejecuta dentro de un hilo y permite capturar la salida del log sin bloquear el notebook.

El log mostrará:
- ID del agente generado automáticamente.
- IP del servidor.
- URL público asignado por NANDA.
- Estado de registro en el `registry` central.


In [27]:
proc = subprocess.Popen(
    [sys.executable, "-u", "/mnt/data/server_nanda.py"],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, env=os.environ.copy()
)

time.sleep(2)
for _ in range(20):
    line = proc.stdout.readline()
    if not line:
        break
    print(line.rstrip())


🤖 NANDA initialized with custom improvement logic: crewai_improvement
🔧 Custom improvement logic 'crewai_improvement' registered
Message improver set to: nanda_custom
✅ AgentBridge created with custom improve_message_direct: crewai_improvement
🌐 Detecting server IP address...
✅ Detected server IP: 34.125.248.107
🤖 Auto-generated agent ID: agents513758
🔗 Auto-generated public URL: http://34.125.248.107:6000
Registry URL file not found. Using default: https://chat.nanda-registry.com:6900
🚀 Starting agent bridge for agents513758 on port 6000...
🚀 NANDA starting agent_bridge server with custom logic...
🔧 UI_CLIENT_URL: https://localhost:6001/api/receive_message
Using default registry URL: https://chat.nanda-registry.com:6900
Registering agent agents513758 with URL http://34.125.248.107:6000 at registry https://chat.nanda-registry.com:6900...

🤖 Agent agents513758 is running
🌐 Server IP: 34.125.248.107
Agent Bridge URL: http://localhost:6000/a2a
Public Client API URL: http://34.125.248.107:

## 🧭 Conclusiones y próximos pasos

El proyecto **Nandito** demuestra la interoperabilidad práctica entre agentes de IA distribuidos mediante **CrewAI** y **NANDA**, utilizando **AWS Bedrock** como base de procesamiento cognitivo.

### Resultados obtenidos:
- Conexión exitosa con AWS Bedrock y ejecución del modelo Claude 3 Haiku.
- Integración funcional entre CrewAI y NANDA Adapter.
- Implementación de un servidor de mensajes accesible vía API REST.

### Próximos pasos:
- Extender el sistema a comunicación multiagente (CrewAI + Soly + NANDA).
- Incorporar una capa de reputación y trazabilidad descentralizada.
- Publicar el agente en una red abierta para pruebas entre universidades o comunidades técnicas.

---
**ODS relacionados:**
- ODS 9 – Industria, innovación e infraestructura  
- ODS 16 – Paz, justicia e instituciones sólidas  
- ODS 17 – Alianzas para lograr los objetivos
